# 데이터 전처리 (25.06.18 오전수업)

## 1. Tidy Data 만들기
- data\pew.csv 활용 
- (미국의 퓨리서치센터의 소득, 종교의 관계를 나타내는 데이터)
- wide data > long data

In [ ]:
# pandas 라이브러리 불러오기 (데이터 분석용)
import pandas as pd

In [ ]:
# CSV 파일을 DataFrame으로 읽어오기
pew_df = pd.read_csv('./data/pew.csv')
# 데이터프레임의 정보 요약 출력
pew_df.info()

In [ ]:
# 데이터프레임의 앞부분 5개 행 출력
pew_df.head()

In [ ]:
pew_df.tail(2)

![image](image/20250618_101949.png)


### long data 로 만들기


In [ ]:
pew_df.head(2)

# melt()

- id_vars : 유지할 변수 리스트(튜플, 행렬)
- value_vars : 피봇 되돌리기 값으로 쓰일 열 목록
- var_name : 만들어지는 새 컬럼의 이름
- value_name : 새 컬럼의 값의 컬럼 이름

In [ ]:
# 열을 행으로 바꾸는 melt (long-form 형태로 변환)
pew_df.melt(id_vars='religion')

In [ ]:
# 열을 행으로 바꾸는 melt (long-form 형태로 변환)
pew_df.melt(id_vars='religion',
            var_name='income',
            value_name='count'
            )

### billboard.csv

### 여러개의 열을 이용한 melt

In [ ]:
# pandas 라이브러리 불러오기 (데이터 분석용)
import pandas as pd
# CSV 파일을 DataFrame으로 읽어오기
billboard_df = pd.read_csv('./data/billboard.csv')
# 데이터프레임의 정보 요약 출력
billboard_df.info()

In [ ]:
billboard_df.head(3)

In [ ]:
id_vars = ['year', 'artist','track', 'time', 'date.entered' ]

# 열을 행으로 바꾸는 melt (long-form 형태로 변환)
b_melt_df = billboard_df.melt(
                  id_vars=id_vars,
                  var_name='week',
                  value_name='rating'
                )
# 데이터프레임의 앞부분 5개 행 출력
b_melt_df.head()

In [ ]:
data_file_name = 'country_timeseries.csv'
# CSV 파일을 DataFrame으로 읽어오기
ebola_df = pd.read_csv('./data/'+data_file_name)
# 데이터프레임의 정보 요약 출력
ebola_df.info()

In [ ]:
ebola_df.iloc[:5, [0,1,2,10]]

In [ ]:
#1. 값 Cases~, Deaths~  id: Date, Day
# 열을 행으로 바꾸는 melt (long-form 형태로 변환)
ebola_long = ebola_df.melt(id_vars=['Date','Day'])
ebola_long

In [ ]:
#2. column 값을 2개로 분할
variable_split = ebola_long['variable'].str.split('_')
type(variable_split)

In [ ]:
variable_split[:5]

In [ ]:
variable_split[0][0]

In [ ]:
status_values = variable_split.str.get(0) #cases, death
country_values = variable_split.str.get(1) #country
type(status_values)

In [ ]:
status_values

In [ ]:
#3. 컬럼 추가
ebola_long['status'] = status_values
ebola_long['country'] = country_values
ebola_long[:5]

### 1~3 번을 빠르게 만들기

In [ ]:
# 1. ebola_df를 긴 형태(long format)로 변환
# 'Date'와 'Day'는 고정시키고, 나머지 컬럼들은 'variable', 'value' 쌍으로 녹인다(melt)
# 열을 행으로 바꾸는 melt (long-form 형태로 변환)
ebola_long_1 = ebola_df.melt(id_vars=['Date','Day'])

# 2. 'variable' 컬럼의 값을 '_'(언더스코어) 기준으로 분리한다.
# 예: 'cases_Guinea' → ['cases', 'Guinea']
variable_split1 = ebola_long_1['variable'].str.split('_', expand=True)

# 3. 분리된 결과를 새로운 컬럼 'status'와 'country'에 나눠서 저장한다.
# 예: 'cases_Guinea' → status = 'cases', country = 'Guinea'
ebola_long_1[['status', 'country']] = variable_split1

# 4. 결과의 앞부분 5개 행을 확인한다.
# 데이터프레임의 앞부분 5개 행 출력
ebola_long_1.head()

### weather.csv 활용

In [ ]:
# pandas 라이브러리 불러오기 (데이터 분석용)
import pandas as pd
# CSV 파일을 DataFrame으로 읽어오기
weather = pd.read_csv('./data/weather.csv')
print(weather.iloc[:5, :11])

In [ ]:
weather.columns

In [ ]:
#1) day 로 언피봇 melt()
# 열을 행으로 바꾸는 melt (long-form 형태로 변환)
weather_melt = weather.melt(
    id_vars=["id", "year", "month", "element"],
    var_name="day", 
    value_name="temp", 
)
print(weather_melt)

In [ ]:
weather_melt.head(2)

## pivot_table()

In [ ]:
#2) element 를 피봇팅 > tmax, tmin 컬럼 생성
# 행을 열로 바꾸는 pivot/pivot_table
weather_tidy = weather_melt.pivot_table(
                  index=['id', 'year', 'month','day'],
                  columns='element',
                  values='temp'
                )
# 데이터프레임의 앞부분 5개 행 출력
weather_tidy.head()

In [ ]:
weather_tidy_flat = weather_tidy.reset_index()
# 데이터프레임의 앞부분 5개 행 출력
weather_tidy_flat.head()

### 메소드 체이닝

In [ ]:
# 행을 열로 바꾸는 pivot/pivot_table
weather_tidy = weather_melt.pivot_table(
                  index=['id', 'year', 'month','day'],
                  columns='element',
                  values='temp'
                ).reset_index()
# 데이터프레임의 앞부분 5개 행 출력
weather_tidy.head()

# 2. apply()

함수를 시리즈, 데이터프레임에 적용하기

1. 사용자 정의 함수 정의

def my_sq(x):
    return x**2

2. 데이터프레임.apply(my_sq)


In [ ]:
def my_sq(x):
    return x ** 2

In [ ]:
# pandas 라이브러리 불러오기 (데이터 분석용)
import pandas as pd

df = pd.DataFrame({"a": [10, 20, 30], 
                   "b": [20, 30, 40]})
df

In [ ]:
dir(df)

In [ ]:
df['a']

In [ ]:
df['a']**2 #벡터화된 연산

In [ ]:
my_sq(10)

In [ ]:
my_sq(df['a'])

In [ ]:
df['a'].apply(my_sq)

In [ ]:
df.iloc[0].apply(my_sq)

In [ ]:
#매개변수 2개 함수 정의
def my_exp(x, e):
    return x ** e

In [ ]:
my_exp(2,3)

In [ ]:
# 함수 적용
df['a'].apply(my_exp, e=3)

### dataframe.apply()

In [68]:
def print_me(x):
  print(x)
  print(type(x))

In [69]:
df

,a,b
0,10,20
1,20,30
2,30,40


In [ ]:
# 함수 적용
df.apply(print_me)

0    10
1    20
2    30
Name: a, dtype: int64
<class 'pandas.core.series.Series'>
0    20
1    30
2    40
Name: b, dtype: int64
<class 'pandas.core.series.Series'>


a    None
b    None
dtype: object

In [ ]:
# 함수 적용
df.apply(print_me, axis=1)

a    10
b    20
Name: 0, dtype: int64
<class 'pandas.core.series.Series'>
a    20
b    30
Name: 1, dtype: int64
<class 'pandas.core.series.Series'>
a    30
b    40
Name: 2, dtype: int64
<class 'pandas.core.series.Series'>


0    None
1    None
2    None
dtype: object

# 월급 계산


In [ ]:
# pandas 라이브러리 불러오기 (데이터 분석용)
import pandas as pd

# 직원 명단 DataFrame 생성
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 45, 35, 32, 28],
    'Salary': [5000, 7000, 6500, 5500, 6000]
}

df = pd.DataFrame(data)
print(df)

      Name  Age  Salary
0    Alice   25    5000
1      Bob   45    7000
2  Charlie   35    6500
3    David   32    5500
4      Eva   28    6000


In [75]:
def age_category(age):
    if age < 30:
        return '20대'
    elif age < 40:
        return '30대'
    else:
        return '40대이상'
    
age_category(20)

'20대'

In [ ]:
# Category 컬럼으로 추가
# 'Age' 컬럼의 각 값을 기준으로 조건 판단 후, 새로운 'Category' 컬럼 생성
# 함수 적용
df["Category"] = df["Age"].apply(

    # 각 나이(age)에 대해 아래 조건을 적용
# lambda: 한 줄짜리 함수 정의
    lambda age: 
        "20대" if 20 <= age < 30     # 나이가 20 이상 30 미만이면 → '20대'
        else "30대" if 30 <= age < 40  # 아니고, 30 이상 40 미만이면 → '30대'
        else "2030세대 X"                    # 그 외는 모두 '2030세대 아님'

)

# 결과 확인
print(df)

      Name  Age  Salary  Category
0    Alice   25    5000       20대
1      Bob   45    7000  2030세대 X
2  Charlie   35    6500       30대
3    David   32    5500       30대
4      Eva   28    6000       20대


In [ ]:
# pandas 라이브러리 불러오기 (데이터 분석용)
import pandas as pd

# 직원 명단 DataFrame 생성
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 45, 35, 32, 28],
    'Salary': [5000, 7000, 6500, 5500, 6000]
}

df = pd.DataFrame(data)

# Salary 기준으로 소득 카테고리 분류
# 함수 적용
df["SalaryGrade"] = df["Salary"].apply(
# lambda: 한 줄짜리 함수 정의
    lambda s: "고소득" if s >= 6000
              else "중간소득" if s >= 5000
              else "저소득"
)

print(df)

      Name  Age  Salary SalaryGrade
0    Alice   25    5000        중간소득
1      Bob   45    7000         고소득
2  Charlie   35    6500         고소득
3    David   32    5500        중간소득
4      Eva   28    6000         고소득


# 3. 데이터 연결/결합하기

### contact() + axis = 

In [ ]:
# CSV 파일을 DataFrame으로 읽어오기
df1 = pd.read_csv('./data/concat_1.csv')#4x4
# CSV 파일을 DataFrame으로 읽어오기
df2 = pd.read_csv('./data/concat_2.csv')#4x4
# CSV 파일을 DataFrame으로 읽어오기
df3 = pd.read_csv('./data/concat_3.csv')#4x4
df1, df2, df3

(    A   B   C   D
 0  a0  b0  c0  d0
 1  a1  b1  c1  d1
 2  a2  b2  c2  d2
 3  a3  b3  c3  d3,
     A   B   C   D
 0  a4  b4  c4  d4
 1  a5  b5  c5  d5
 2  a6  b6  c6  d6
 3  a7  b7  c7  d7,
      A    B    C    D
 0   a8   b8   c8   d8
 1   a9   b9   c9   d9
 2  a10  b10  c10  d10
 3  a11  b11  c11  d11)

### column 이 동일해야 한다.

In [81]:
total_df = pd.concat([df1, df2, df3])
total_df

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7
0,a8,b8,c8,d8
1,a9,b9,c9,d9


In [82]:
total_df.loc[3]

,A,B,C,D
3,a3,b3,c3,d3
3,a7,b7,c7,d7
3,a11,b11,c11,d11


In [83]:
total_df.reset_index()

,index,A,B,C,D
0,0,a0,b0,c0,d0
1,1,a1,b1,c1,d1
2,2,a2,b2,c2,d2
3,3,a3,b3,c3,d3
4,0,a4,b4,c4,d4
5,1,a5,b5,c5,d5
6,2,a6,b6,c6,d6
7,3,a7,b7,c7,d7
8,0,a8,b8,c8,d8
9,1,a9,b9,c9,d9


In [84]:
total_df = pd.concat([df1, df2, df3], ignore_index=True)
total_df

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5
6,a6,b6,c6,d6
7,a7,b7,c7,d7
8,a8,b8,c8,d8
9,a9,b9,c9,d9


In [85]:
add_s = pd.Series(['a100', 'b100', 'c100', 'd100'])
type(add_s)

pandas.core.series.Series

In [86]:
pd.concat([df1,add_s ])

,A,B,C,D,0
0,a0,b0,c0,d0,NaN
1,a1,b1,c1,d1,NaN
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,NaN
0,NaN,NaN,NaN,NaN,a100
1,NaN,NaN,NaN,NaN,b100
2,NaN,NaN,NaN,NaN,c100
3,NaN,NaN,NaN,NaN,d100


In [87]:
add_s = pd.Series(
  data=['a100', 'b100', 'c100', 'd100'],
  index=['A','B','C','D']
)
add_s

A    a100
B    b100
C    c100
D    d100
dtype: object

**concat() 의 기준은 columns 값이다.**  
**연결하려는 데이터프레임의 columns 가 동일해야 한다.**

In [88]:
empty_df = pd.DataFrame(columns=['A','B','C','D'])
empty_df.loc[0] = add_s
empty_df

,A,B,C,D
0,a100,b100,c100,d100


In [89]:
pd.concat([df1, empty_df])

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a100,b100,c100,d100


In [92]:
new_row = pd.DataFrame(data = [['a100', 'b100', 'c100', 'd100']],
             columns=['A','B','C','D'])
new_row

,A,B,C,D
0,a100,b100,c100,d100


In [93]:
pd.concat([df1, new_row])

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a100,b100,c100,d100


# 3-2. 열 연결하기
- concat( [ ] , axis=1) #columns

In [94]:
col_concat = pd.concat([df1,df2,df3], axis='columns')
col_concat['A']

,A,A,A
0,a0,a4,a8
1,a1,a5,a9
2,a2,a6,a10
3,a3,a7,a11


In [95]:
#col 추가
#col의 len 이 일치
col_concat['new_col'] = ['n1', 'n2', 'n3', 'n4'] 
col_concat

,A,B,C,D,A,B,C,D,A,B,C,D,new_col
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8,n1
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9,n2
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10,n3
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11,n4


In [96]:
#ignore_index=True
#series 추가

In [97]:
col_concat_1 = pd.concat([df1,df2,df3], 
                       axis='columns',
                       ignore_index=True)
col_concat_1

,0,1,2,3,4,5,6,7,8,9,10,11
0,a0,b0,c0,d0,a4,b4,c4,d4,a8,b8,c8,d8
1,a1,b1,c1,d1,a5,b5,c5,d5,a9,b9,c9,d9
2,a2,b2,c2,d2,a6,b6,c6,d6,a10,b10,c10,d10
3,a3,b3,c3,d3,a7,b7,c7,d7,a11,b11,c11,d11


# 3-3. 병합
- concat(join='inner'/'outer')

In [100]:
df1.columns = ['A', 'B', 'C', 'D']
df2.columns = ['E', 'F', 'G', 'H']
df3.columns = ['A', 'C', 'F', 'H'] 

In [99]:
pd.concat([df1, df2, df3])


,A,B,C,D,E,F,G,H
0,a0,b0,c0,d0,NaN,NaN,NaN,NaN
1,a1,b1,c1,d1,NaN,NaN,NaN,NaN
2,a2,b2,c2,d2,NaN,NaN,NaN,NaN
3,a3,b3,c3,d3,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,a4,b4,c4,d4
1,NaN,NaN,NaN,NaN,a5,b5,c5,d5
2,NaN,NaN,NaN,NaN,a6,b6,c6,d6
3,NaN,NaN,NaN,NaN,a7,b7,c7,d7
0,a8,NaN,b8,NaN,NaN,c8,NaN,d8
1,a9,NaN,b9,NaN,NaN,c9,NaN,d9


In [101]:
pd.concat([df1, df2, df3], join='inner').shape


(12, 0)

In [102]:
pd.concat([df1, df3], join='inner')


,A,C
0,a0,c0
1,a1,c1
2,a2,c2
3,a3,c3
0,a8,b8
1,a9,b9
2,a10,b10
3,a11,b11


### 열방향으로 병합 > index


In [103]:
df1.index = [0, 1, 2, 3]
df2.index = [4, 5, 6, 7]
df3.index = [0, 2, 5, 7]

In [104]:
pd.concat([df1,df2,df3], axis='columns')


,A,B,C,D,E,F,G,H,A,C,F,H
0,a0,b0,c0,d0,NaN,NaN,NaN,NaN,a8,b8,c8,d8
1,a1,b1,c1,d1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a2,b2,c2,d2,NaN,NaN,NaN,NaN,a9,b9,c9,d9
3,a3,b3,c3,d3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,a4,b4,c4,d4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,a5,b5,c5,d5,a10,b10,c10,d10
6,NaN,NaN,NaN,NaN,a6,b6,c6,d6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,a7,b7,c7,d7,a11,b11,c11,d11


In [105]:
pd.concat([df1,df2,df3], axis='columns', join='inner')


,A,B,C,D,E,F,G,H,A,C,F,H


### 열병합 연습


In [111]:
import pandas as pd

# 월별 매출 데이터
january = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-02'],
    'Sales': [200, 250]
})

february = pd.DataFrame({
    'Date': ['2023-02-01', '2023-02-02'],
    'Sales': [300, 350]
})

In [112]:
# 상점 A의 매출 데이터
store_a = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-02'],
    'Sales_A': [200, 250]
})

# 상점 B의 매출 데이터
store_b = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-02'],
    'Sales_B': [300, 350]
})

In [113]:
# 상점 C의 매출 데이터
store_c = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-03'],
    'Sales_C': [220, 280]
})

# 상점 D의 매출 데이터
store_d = pd.DataFrame({
    'Date': ['2023-01-02', '2023-01-03'],
    'Sales_D': [330, 370]
})

# 파일로 결합하기 25.06.18 (오후수업)

| 3. 데이터 결합하기 | .concat(), .merge(), |  |


In [131]:
# 상대경로 방식
from pathlib import Path

data_files = Path("./").glob("data/billboard-by_week/billboard-*.csv")

In [132]:
b_data_files = sorted(list(data_files))
len(b_data_files)

76

In [133]:
[b_data_file for b_data_file in b_data_files]


[WindowsPath('data/billboard-by_week/billboard-01.csv'),
 WindowsPath('data/billboard-by_week/billboard-02.csv'),
 WindowsPath('data/billboard-by_week/billboard-03.csv'),
 WindowsPath('data/billboard-by_week/billboard-04.csv'),
 WindowsPath('data/billboard-by_week/billboard-05.csv'),
 WindowsPath('data/billboard-by_week/billboard-06.csv'),
 WindowsPath('data/billboard-by_week/billboard-07.csv'),
 WindowsPath('data/billboard-by_week/billboard-08.csv'),
 WindowsPath('data/billboard-by_week/billboard-09.csv'),
 WindowsPath('data/billboard-by_week/billboard-10.csv'),
 WindowsPath('data/billboard-by_week/billboard-11.csv'),
 WindowsPath('data/billboard-by_week/billboard-12.csv'),
 WindowsPath('data/billboard-by_week/billboard-13.csv'),
 WindowsPath('data/billboard-by_week/billboard-14.csv'),
 WindowsPath('data/billboard-by_week/billboard-15.csv'),
 WindowsPath('data/billboard-by_week/billboard-16.csv'),
 WindowsPath('data/billboard-by_week/billboard-17.csv'),
 WindowsPath('data/billboard-by

In [ ]:
# 절대경로 방식

from pathlib import Path

# 상대 경로 기준 경로 지정
target_path = Path("data/billboard-by_week")

# glob + 정렬 → 바로 b_data_files에 저장
b_data_files = sorted(list(target_path.glob("billboard-*.csv")))

# 결과 확인
for file in b_data_files:
    print(file)


data\billboard-by_week\billboard-01.csv
data\billboard-by_week\billboard-02.csv
data\billboard-by_week\billboard-03.csv
data\billboard-by_week\billboard-04.csv
data\billboard-by_week\billboard-05.csv
data\billboard-by_week\billboard-06.csv
data\billboard-by_week\billboard-07.csv
data\billboard-by_week\billboard-08.csv
data\billboard-by_week\billboard-09.csv
data\billboard-by_week\billboard-10.csv
data\billboard-by_week\billboard-11.csv
data\billboard-by_week\billboard-12.csv
data\billboard-by_week\billboard-13.csv
data\billboard-by_week\billboard-14.csv
data\billboard-by_week\billboard-15.csv
data\billboard-by_week\billboard-16.csv
data\billboard-by_week\billboard-17.csv
data\billboard-by_week\billboard-18.csv
data\billboard-by_week\billboard-19.csv
data\billboard-by_week\billboard-20.csv
data\billboard-by_week\billboard-21.csv
data\billboard-by_week\billboard-22.csv
data\billboard-by_week\billboard-23.csv
data\billboard-by_week\billboard-24.csv
data\billboard-by_week\billboard-25.csv


In [ ]:
b_01 = pd.read_csv(b_data_files[0])
b_01

In [134]:
list_b_df = [] #전체파일

for b_data_file in b_data_files:
  df = pd.read_csv(b_data_file)
  list_b_df.append(df)

len(list_b_df)

76

In [137]:
type(list_b_df[0])


pandas.core.frame.DataFrame

In [136]:
list_b_df[1].head()


,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk2,82.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk2,87.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk2,70.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk2,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk2,34.0


In [135]:
print(type(list_b_df))   # 리스트인지 먼저 확인
print(len(list_b_df))    # 리스트 길이 (0이면 비어있음)
print(list_b_df)         # 실제 내용


<class 'list'>
76
[     year            artist                    track  time date.entered week  \
0    2000             2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26  wk1   
1    2000           2Ge+her  The Hardest Part Of ...  3:15   2000-09-02  wk1   
2    2000      3 Doors Down               Kryptonite  3:53   2000-04-08  wk1   
3    2000      3 Doors Down                    Loser  4:24   2000-10-21  wk1   
4    2000          504 Boyz            Wobble Wobble  3:35   2000-04-15  wk1   
..    ...               ...                      ...   ...          ...  ...   
312  2000       Yankee Grey     Another Nine Minutes  3:10   2000-04-29  wk1   
313  2000  Yearwood, Trisha          Real Live Woman  3:55   2000-04-01  wk1   
314  2000   Ying Yang Twins  Whistle While You Tw...  4:19   2000-03-18  wk1   
315  2000     Zombie Nation            Kernkraft 400  3:30   2000-09-02  wk1   
316  2000   matchbox twenty                     Bent  4:12   2000-04-29  wk1   

     rating  
0     

In [138]:
final_df = pd.concat(list_b_df)
final_df.shape

(24092, 7)

# 데이터전처리 
| 3. 데이터 결합하기 | .concat(), .merge(), |  |



### merge()

In [143]:
# 매출 데이터
sales_data = pd.DataFrame({
    'CustomerID': [101, 102, 103, 104],
    'Date': ['2023-01-01', '2023-01-03', '2023-01-05', '2023-01-07'],
    'Amount': [250, 300, 150, 400]
})

# 고객 데이터
customer_data = pd.DataFrame({
    'CustomerID': [101, 102, 103, 105],
    'Name': ['Alice', 'Bob', 'Charlie', 'Eva'],
    'Region': ['North', 'South', 'East', 'West']
})


In [140]:
sales_data

,CustomerID,Date,Amount
0,101,2023-01-01,250
1,102,2023-01-03,300
2,103,2023-01-05,150
3,104,2023-01-07,400


In [141]:
customer_data

,CustomerID,Name,Region
0,101,Alice,North
1,102,Bob,South
2,103,Charlie,East
3,105,Eva,West


In [147]:
pd.merge(sales_data, customer_data, on="CustomerID") #inner 104랑 105는 공통부분이 아니라서 안나옴.

,CustomerID,Date,Amount,Name,Region
0,101,2023-01-01,250,Alice,North
1,102,2023-01-03,300,Bob,South
2,103,2023-01-05,150,Charlie,East


In [148]:
pd.merge(sales_data, customer_data, on="CustomerID", how='outer')

,CustomerID,Date,Amount,Name,Region
0,101,2023-01-01,250.0,Alice,North
1,102,2023-01-03,300.0,Bob,South
2,103,2023-01-05,150.0,Charlie,East
3,104,2023-01-07,400.0,NaN,NaN
4,105,NaN,NaN,Eva,West


In [ ]:
pd.merge(sales_data, customer_data, on="CustomerID", how='left') # left sales 기준으로됨

,CustomerID,Date,Amount,Name,Region
0,101,2023-01-01,250,Alice,North
1,102,2023-01-03,300,Bob,South
2,103,2023-01-05,150,Charlie,East
3,104,2023-01-07,400,NaN,NaN


In [152]:
pd.merge(sales_data, customer_data, on="CustomerID", how='right') # right customer 기준으로됨

,CustomerID,Date,Amount,Name,Region
0,101,2023-01-01,250.0,Alice,North
1,102,2023-01-03,300.0,Bob,South
2,103,2023-01-05,150.0,Charlie,East
3,105,NaN,NaN,Eva,West


# 4. 그룹 연산하기

- groupby(), aggregation, 집계



In [158]:
import pandas as pd

# 1. gapminder.tsv 불러오기
df = pd.read_csv("data/gapminder.tsv", sep='\t')

# 2. 고유 국가 수 확인
print(df['country'].unique().shape)

# 3. 국가별 평균 기대수명
print(df.groupby('country')['lifeExp'].mean())


(142,)
country
Afghanistan           37.478833
Albania               68.432917
Algeria               59.030167
Angola                37.883500
Argentina             69.060417
                        ...    
Vietnam               57.479500
West Bank and Gaza    60.328667
Yemen, Rep.           46.780417
Zambia                45.996333
Zimbabwe              52.663167
Name: lifeExp, Length: 142, dtype: float64


In [159]:
df['country'].unique().shape

(142,)

In [160]:
df.groupby('country')['lifeExp'].mean()

country
Afghanistan           37.478833
Albania               68.432917
Algeria               59.030167
Angola                37.883500
Argentina             69.060417
                        ...    
Vietnam               57.479500
West Bank and Gaza    60.328667
Yemen, Rep.           46.780417
Zambia                45.996333
Zimbabwe              52.663167
Name: lifeExp, Length: 142, dtype: float64

count(),
size(),
mean(),
std(),
quantile(),
max(), min(), sum(), var(), describe()

### numpy 등도 적용 가능하다
np.max(), np.sum(), np.var()

In [164]:
df.groupby('continent')['lifeExp'].count()


continent
Africa      624
Americas    300
Asia        396
Europe      360
Oceania      24
Name: lifeExp, dtype: int64

# 여러개를 적용해서 합할 때 agg()

In [168]:
import numpy as np

df.groupby('continent')['lifeExp'].agg([np.mean, np.std, np.min, np.max])



C:\Users\Admin\AppData\Local\Temp\ipykernel_17476\3497464177.py:3: FutureWarning: The provided callable <function mean at 0x00000218FF073D80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby('continent')['lifeExp'].agg([np.mean, np.std, np.min, np.max])
C:\Users\Admin\AppData\Local\Temp\ipykernel_17476\3497464177.py:3: FutureWarning: The provided callable <function std at 0x00000218FF073EC0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  df.groupby('continent')['lifeExp'].agg([np.mean, np.std, np.min, np.max])
C:\Users\Admin\AppData\Local\Temp\ipykernel_17476\3497464177.py:3: FutureWarning: The provided callable <function min at 0x00000218FF0734C0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided 

,mean,std,min,max
continent,,,,
Africa,48.865330,9.150210,23.599,76.442
Americas,64.658737,9.345088,37.579,80.653
Asia,60.064903,11.864532,28.801,82.603
Europe,71.903686,5.433178,43.585,81.757
Oceania,74.326208,3.795611,69.120,81.235


In [176]:
df.groupby('continent')['lifeExp'].agg([
    ('평균', np.mean),
    ('표준편차', np.std),
    ('최소값', np.min),
    ('최대값', np.max)
])


C:\Users\Admin\AppData\Local\Temp\ipykernel_17476\4133836837.py:1: FutureWarning: The provided callable <function mean at 0x00000218FF073D80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby('continent')['lifeExp'].agg([
C:\Users\Admin\AppData\Local\Temp\ipykernel_17476\4133836837.py:1: FutureWarning: The provided callable <function std at 0x00000218FF073EC0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  df.groupby('continent')['lifeExp'].agg([
C:\Users\Admin\AppData\Local\Temp\ipykernel_17476\4133836837.py:1: FutureWarning: The provided callable <function min at 0x00000218FF0734C0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the 

,평균,표준편차,최소값,최대값
continent,,,,
Africa,48.865330,9.150210,23.599,76.442
Americas,64.658737,9.345088,37.579,80.653
Asia,60.064903,11.864532,28.801,82.603
Europe,71.903686,5.433178,43.585,81.757
Oceania,74.326208,3.795611,69.120,81.235


In [180]:
agg_df = df.groupby('continent')\
    .agg({'lifeExp':'mean', 'pop':'median','gdpPercap':'median'}) 
agg_df

,lifeExp,pop,gdpPercap
continent,,,
Africa,48.865330,4579311.0,1192.138217
Americas,64.658737,6227510.0,5465.509853
Asia,60.064903,14530830.5,2646.786844
Europe,71.903686,8551125.0,12081.749115
Oceania,74.326208,6403491.5,17983.303955
